In [1]:
!python --version
!pyspark --version

Python 3.7.12
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.4.0
      /_/
                        
Using Scala version 2.11.12, OpenJDK 64-Bit Server VM, 1.8.0_312
Branch 
Compiled by user  on 2018-10-29T06:22:05Z
Revision 
Url 
Type --help for more information.


In [2]:
# set settings here
BUCKET = "tcceval-data"
DATABASE_CACHE_FOLDER = "_db_cache"
DUMP_PARQUET_FOLDER = "craigslist_parquet"
DUMP_EXCEL_FOLDER = "craigslist_excel"

In [3]:
# Establishes Spark session
ADDITIONAL_ARTIFACTS = [
    # Apache Sedona handles spatial queries
    'org.apache.sedona:sedona-python-adapter-2.4_2.11:1.2.0-incubating',
    'org.apache.sedona:sedona-viz-2.4_2.11:1.2.0-incubating',
    'org.datasyslab:geotools-wrapper:1.1.0-25.2',
    # JDBC connector for MySQL
    'mysql:mysql-connector-java:8.0.30'
]

import sagemaker_pyspark
from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

classpath = ":".join(sagemaker_pyspark.classpath_jars())
print(classpath)
spark = SparkSession.builder \
    .config("spark.driver.extraClassPath", classpath) \
    .config("spark.executor.extraClassPath", classpath) \
    .config("spark.serializer", KryoSerializer.getName) \
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
    .config('spark.jars.packages',
           ','.join(ADDITIONAL_ARTIFACTS)) \
    .getOrCreate()
SedonaRegistrator.registerAll(spark)
# suppresses a bunch of AbortableS3Stream warnings
spark.sparkContext.setLogLevel("ERROR")

/home/ec2-user/bash/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-core-1.11.835.jar:/home/ec2-user/bash/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-kms-1.11.835.jar:/home/ec2-user/bash/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-s3-1.11.835.jar:/home/ec2-user/bash/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-sagemaker-1.11.835.jar:/home/ec2-user/bash/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-sagemakerruntime-1.11.835.jar:/home/ec2-user/bash/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-sts-1.11.835.jar:/home/ec2-user/bash/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/hadoop-annotations-2.8.1.jar:/home/ec2-user/bash/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/hadoop-auth-2.8.1.jar:/home/ec2-user/bash/envs/spark_2_4_0/lib/pytho

Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
:: loading settings :: url = jar:file:/home/ec2-user/bash/envs/spark_2_4_0/lib/python3.7/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.sedona#sedona-python-adapter-2.4_2.11 added as a dependency
org.apache.sedona#sedona-viz-2.4_2.11 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c6d7a0c3-4411-48dd-8f1b-b68c9d0b1206;1.0
	confs: [default]
	found org.apache.sedona#sedona-python-adapter-2.4_2.11;1.2.0-incubating in central
	found org.locationtech.jts#jts-core;1.18.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found com.fasterxml.jackson.core#jackson-databind;2.12.2 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.12.2 in central
	found com.fasterxml.

2022-08-05 18:11:11 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-08-05 18:11:12 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-08-05 18:11:12 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
%%time
# Est. 50 sec
# load in the craigslist data
craigslist = spark.read.parquet(f"s3a://{BUCKET}/{DUMP_PARQUET_FOLDER}/")
craigslist.printSchema()
craigslist.createOrReplaceTempView("craigslist")
craigslist.show(n=5)

root
 |-- pid: long (nullable = true)
 |-- repostid: string (nullable = true)
 |-- dt: timestamp (nullable = true)
 |-- url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- price: double (nullable = true)
 |-- neighb: string (nullable = true)
 |-- beds: long (nullable = true)
 |-- sqft: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- accuracy: double (nullable = true)
 |-- address: string (nullable = true)
 |-- posttext: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- group: long (nullable = true)
 |-- region: string (nullable = true)
 |-- geoid: string (nullable = true)
 |-- url_type: string (nullable = true)
 |-- is_adu: boolean (nullable = true)
 |-- type: string (nullable = true)
 |-- pct_price: double (nullable = true)
 |-- pct_sqft: double (nullable = true)

+----------+----------+-------------------+--------------------+--------------------+------+--------------------+----+----+---------+---

In [18]:
# prepare dataframe by adding the following columns:
# - time: 0 for <3/2020, 1 for > 9/2021
# - post_treament, which is time*group
# we also want to clear the bottom 2% and top 2% from the dataset in both sqft and rent
import pyspark.sql.functions as F

LOW_CUTOFF = F.to_date(F.lit('2020-03-01'))
HI_CUTOFF = F.to_date(F.lit('2021-09-01'))

did = craigslist.where((craigslist['pct_price'] > 0.02) & (craigslist['pct_price'] < 0.98) & (craigslist['pct_sqft'] > 0.02) & (craigslist['pct_sqft'] < 0.98))
did = did.withColumn('time', F.when(did['dt'] < LOW_CUTOFF, 0) \
                            .when(did['dt'] > HI_CUTOFF, 1) \
                            .otherwise(-1))
# get rid of the -1's in time column
did = did.where(did['time'] != -1)
# add post_streatment
did = did.withColumn('post_treatment', did['time'] * did['group'])

# now group by geoid
did.createOrReplaceTempView("did")
did = spark.sql(
    """
    SELECT percentile(price, 0.5) as med_price, percentile(sqft, 0.5) as med_sqft, FIRST(region) as region,
        geoid, time, group, post_treatment
    FROM did
    GROUP BY geoid, time, group, post_treatment
    """)
did = did.withColumn("med_price_per_sqft", did['med_price'] / did['med_sqft'])
did.printSchema()
did.explain()

root
 |-- med_price: double (nullable = true)
 |-- med_sqft: double (nullable = true)
 |-- region: string (nullable = true)
 |-- geoid: string (nullable = true)
 |-- time: integer (nullable = false)
 |-- group: long (nullable = true)
 |-- post_treatment: long (nullable = true)
 |-- med_price_per_sqft: double (nullable = true)

== Physical Plan ==
*(2) Project [med_price#1363, med_sqft#1364, region#1366, geoid#28, time#1312, group#26L, post_treatment#1337L, (med_price#1363 / med_sqft#1364) AS med_price_per_sqft#1376]
+- ObjectHashAggregate(keys=[geoid#28, time#1312, group#26L, post_treatment#1337L], functions=[percentile(price#16, 0.5, 1, 0, 0), percentile(sqft#19L, 0.5, 1, 0, 0), first(region#27, false)])
   +- Exchange hashpartitioning(geoid#28, time#1312, group#26L, post_treatment#1337L, 200)
      +- ObjectHashAggregate(keys=[geoid#28, time#1312, group#26L, post_treatment#1337L], functions=[partial_percentile(price#16, 0.5, 1, 0, 0), partial_percentile(sqft#19L, 0.5, 1, 0, 0), parti

In [19]:
%%time
did = did.toPandas()

[Stage 7:===============================================>       (174 + 4) / 200]

CPU times: user 10.6 ms, sys: 2.61 ms, total: 13.2 ms
Wall time: 2.67 s


In [21]:
import statsmodels.api as sm
predictors = ['time', 'group', 'post_treatment']
response = 'med_price_per_sqft'

# create design matrix and response vector
X = did[predictors]
y = did[response]

# estimate a simple linear regression model with OLS, using statsmodels
model = sm.OLS(y, sm.add_constant(X))
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:     med_price_per_sqft   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.850
Date:                Fri, 05 Aug 2022   Prob (F-statistic):              0.138
Time:                        18:40:00   Log-Likelihood:                -1717.2
No. Observations:                 296   AIC:                             3442.
Df Residuals:                     292   BIC:                             3457.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             14.9271      7.718      1.